In [65]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

## Load datasets

In [66]:
combined_data = pd.read_csv("real_data.csv", sep=",")
synthetic_data_sdv = pd.read_csv("synthetic_data_sdv.csv", sep=",")
synthetic_data_gan = pd.read_csv("synthetic_data_gan.csv", sep=",")

## Take lowest demand value

In [67]:
# Group by 'source' and 'target', then get the minimum 'demand_value' for each group
min_demand_real = combined_data.loc[combined_data.groupby(['source', 'target'])['demand_value'].idxmin()]
min_demand_gan = synthetic_data_gan.loc[synthetic_data_gan.groupby(['source', 'target'])['demand_value'].idxmin()]
min_demand_sdv = synthetic_data_sdv.loc[synthetic_data_sdv.groupby(['source', 'target'])['demand_value'].idxmin()]

In [68]:
# Filter out rows with same source and destination
min_demand_gan = min_demand_gan[min_demand_gan["source"] != min_demand_gan["target"]]
min_demand_sdv = min_demand_sdv[min_demand_sdv["source"] != min_demand_sdv["target"]]

## Build graph 

In [69]:
def create_graph(data):
    G = nx.Graph()
    for index, row in data.iterrows():
        G.add_edge(row['source'], row['target'], weight=row['demand_value'])
    return G

In [70]:
def plot_graph(G):
    # Define layout for the graph (spring layout for better spacing)
    pos = nx.spring_layout(G, seed=42)

    # Draw nodes and edges
    plt.figure(figsize=(12, 8))
    nx.draw_networkx_nodes(G, pos, node_size=500, node_color='skyblue', edgecolors='k')
    nx.draw_networkx_edges(G, pos, width=1.5, alpha=0.7, edge_color='gray')

    # Add labels for nodes
    nx.draw_networkx_labels(G, pos, font_size=10, font_color='black')

    # Add edge weights as labels
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8, font_color='red')

    # Customize plot appearance
    plt.title("Network Graph with Shortest Path Demand Values", fontsize=16)
    plt.axis('off')
    plt.show()  
    

## Find the shortest path from a source to a destination

In [71]:
def calculate_shortest_path_load(G, source, target):
    # Find the shortest path using Dijkstra's algorithm (based on demand values)
    path = nx.shortest_path(G, source=source, target=target, weight='weight')
    total_load = sum(G[u][v]['weight'] for u, v in zip(path[:-1], path[1:]))
    return path, total_load

## Testing

In [72]:
# Create graphs
G_real = create_graph(min_demand_real)
G_gan = create_graph(min_demand_gan)
G_sdv = create_graph(min_demand_sdv)

In [ ]:
# Source and target nodes
source_node = "KSCYng"
target_node = "HSTNng"

# Calculate and print shortest path loads
for graph_name, graph in [("Real Data", G_real), ("GAN Synthetic Data", G_gan), ("SDV Synthetic Data", G_sdv)]:
    path, total_load = calculate_shortest_path_load(graph, source_node, target_node)
    if path:
        print(f"{graph_name}: Shortest path from {source_node} to {target_node} is {path} with a total load of {total_load:.2f}")
    else:
        print(f"{graph_name}: No path exists from {source_node} to {target_node}")